# comercio_ext_auxiliares.tb_auxiliar_unidades_federativas
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/UF_delta`
## 📌 Descrição do arquivo
Referência de **UF** (unidades federativas) e região.
|Coluna|Descrição|
|---|---|
|`CO_UF`|Código da UF|
|`SG_UF`|Sigla da UF|
|`NO_UF`|Nome da UF|
|`NO_REGIAO`|Nome da região|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:
val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/UF_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_auxiliares/tb_auxiliar_unidades_federativas/"   
val silverTable =    "tb_auxiliar_unidades_federativas"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_UF" , StringType ,  nullable = false ) ,
      StructField ( "SG_UF" , StringType ,  nullable = false ) ,
      StructField ( "NO_UF" , StringType ,  nullable = false ) ,
      StructField ( "NO_REGIAO" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_UF", trim(upper(col("CO_UF").cast(StringType))))
  .withColumn("SG_UF", trim(upper(col("SG_UF").cast(StringType))))
  .withColumn("NO_UF", col("NO_UF").cast(StringType))
  .withColumn("NO_REGIAO", col("NO_REGIAO").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/UF_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_UF").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_UF"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_UF"), col("SG_UF"), col("NO_UF"), col("NO_REGIAO"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}